# 1번 방법 : 블로그에 준비해둔 데이터를 url통해서 가져오기.
## 문제점1 : 블로그는 javascript 기반이라 selenium을 이용해야 한다고 합니다.
## 문제점2 : selenium 을 사용한다고 해도 데이터 테이블 자체가 tr, td 로 가지런히 배열된 느낌이 아니에요 초콜릿 판 처럼 독립된 틀이 많아서 이것도 제가 selector 지정을 잘 이해 못하는 문제 같아요

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

# ChromeDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://blog.naver.com/hellocube/223615712262'
driver.get(url)

# 페이지가 로드될 시간을 주기 위해 잠시 대기
time.sleep(3)  # 필요한 경우 대기 시간 조정

# CSV 파일로 저장
with open('table_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # 특정 ID를 가진 span 요소들 선택
    span_elements = driver.find_elements(By.CSS_SELECTOR, 'span[id^="SE-"]')  # "SE-"로 시작하는 모든 span 요소 선택
    
    for element in span_elements:
        value = element.text.strip()  # 값 추출
        writer.writerow([value])  # CSV에 값 저장

# 브라우저 닫기
driver.quit()

print("CSV 파일로 저장되었습니다.")

# 2번 방법 : 교통정보 사이트에서 url 찾기
## 문제점 : url 특정이 어려운 상태. F12누른 후, 네트워크 창에서 url 정보를 찾을 수 있다고 하는데 잘 못찾겠어요
### : url만 찾을 수 있다면 selector 찾은 후 코드 작성이 가능할 듯 합니다.

In [1]:
import requests
from bs4 import BeautifulSoup
import csv

# URL 설정
url = 'https://taas.koroad.or.kr/sta/acs/exs/selectPatternInfo.do?subjectId=138&patternId=149&termsYn=Y&_ts=1012%7C0719%7C774'

# 웹 페이지 요청
res = requests.get(url)

# 응답 페이지에서 필요한 정보 추출
if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # 표 데이터 추출
    table = soup.find('table')
    table_rows = table.select('tbody tr')  # tbody 내의 모든 행 선택

    # CSV 파일로 저장
    with open('table_data.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        for row in table_rows:
            cells = row.select('td')  # 데이터 셀 선택
            cell_data = [cell.get_text(strip=True) for cell in cells]  # 텍스트 추출
            writer.writerow(cell_data)

    print("CSV 파일로 저장되었습니다.")
else:
    print(f"요청 실패. 응답 코드: {res.status_code}")


AttributeError: 'NoneType' object has no attribute 'select'

# 3번 방법 : 외부 통계표를 특정 url로 찾았습니다 현재 방법으로는 가장 가능성이 있어보여요
### span class = 'val' 인 값으로 찾으면 좋을 것 같아요
### 현재 selector는 #mainTable > tbody > tr:nth-child(9) > td:nth-child(3) > span.val 중에 tr 부분까지 라고 생각합니다.


In [5]:
import requests
from bs4 import BeautifulSoup
import csv

# URL 설정
url = 'https://stat.eseoul.go.kr/statHtml/statHtml.do?orgId=201&tblId=DT_201004_O100001&conn_path=I2'

# 웹 페이지 요청
response = requests.get(url)

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# span class='val' 요소 찾기
values = soup.find_all('span', class_='val')

# CSV 파일로 저장
with open('data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Value'])  # 헤더 작성
    
    for value in values:
        writer.writerow([value.text])  # 각 값을 한 행에 작성

print("Data has been saved to data.csv")


Data has been saved to data.csv
